# Rhadamanthys
> What is this thing are we just looking at a downloader

- toc: true 
- badges: true
- categories: [rhadamanthys,config,IDA,shifted pointers,PEB,_LIST_ENTRY,_LDR_DATA_TABLE_ENTRY]


## Overview

### Sample

`dca16a0e7bdc4968f1988c2d38db133a0e742edf702c923b4f4a3c2f3bdaacf5` [Malware Bazaar](https://bazaar.abuse.ch/sample/dca16a0e7bdc4968f1988c2d38db133a0e742edf702c923b4f4a3c2f3bdaacf5/)

### References
- [Triage Run](https://tria.ge/221226-1ep5eadg74/behavioral2#report)
- [Dancing With Shellcodes: Analyzing Rhadamanthys Stealer](https://elis531989.medium.com/dancing-with-shellcodes-analyzing-rhadamanthys-stealer-3c4986966a88)
  - Downloader `af04ee03d69a7962fa5350d0df00fafc4ae85a07dff32f99f0d8d63900a47466`
 

## Stage 1
The first stage is a C++ loader that is used to decrypt, load, and execute the 2nd stage shellcode. The program flow of the loader is difficult to follow due to the C++ and some nested structures and callbacks. It is not clear if this was intentional or note. 

The shellcode appears to be located in what appears to be a giant Base64 encoded string (not confirmed). This first stage may have alos implmented some PAGE_GUARD exception handling progrem flow redirection for anti-analysis (not confrimed). Dispite this it is trivial to execute the stage until it jumps to the decrypted loaded shellcode (as long as the PAGE_GUARD exceptions are passed to the process).

## Stage 2

For analysis we just dumped the entire memory region that contained the second stage shellcode. The region is based at `0x00760000` but the shellcode entry is at `0x00780A68`. The memory region dump is available on Malshare [`d4f37699c4b283418d1c73416436826e95858cf07f3c29e6af76e91db98e0fc0`](https://malshare.com/sample.php?action=detail&hash=d4f37699c4b283418d1c73416436826e95858cf07f3c29e6af76e91db98e0fc0).

The first task of the shellcode is to walk the PEB to locate Kernel32.

### PEB Walk _LDR_DATA_TABLE_ENTRY and Shifted Pointers in IDA

The process of "walking the PEB" to access the modules loaded in a process as been a shellcode meme since the beginning of shellcode. It's everywhere, and we all mostly understand how it works. 

The issue comes when we try to represent this cleanly in IDA's pseudocode view... accessing a `_LDR_DATA_TABLE_ENTRY` via its `LIST_ENTRY` in the `_PEB_LDR_DATA` structure creates a very messy IDB. Instead of the `_LDR_DATA_TABLE_ENTRY` members we see unhelpful offsets relative to the Flink and Blink member of the `LIST_ENTRY`.

![](https://i.imgur.com/Ta8hGnm.png)

The reason this happens is that the `InMemoryOrderModuleList` is defined as a `LIST_ENTRY` in the `_PEB_LDR_DATA`. The `LIST_ENTRY` struct simply describes a linked list as follows.

```C
typedef struct _LIST_ENTRY {
   struct _LIST_ENTRY *Flink;
   struct _LIST_ENTRY *Blink;
} 
```

The problem comes from where that struct is actually located in the `_LDR_DATA_TABLE_ENTRY`...

```C
typedef struct _LDR_DATA_TABLE_ENTRY {
    PVOID Reserved1[2];
    LIST_ENTRY InMemoryOrderLinks; //<---- Offset into the struct!
    PVOID Reserved2[2];
    PVOID DllBase;
    PVOID EntryPoint;
    PVOID Reserved3;
    UNICODE_STRING FullDllName;
    BYTE Reserved4[8];
    PVOID Reserved5[3];
    union {
        ULONG CheckSum;
        PVOID Reserved6;
    };
    ULONG TimeDateStamp;
}
```

Because the `LIST_ENTRY` is offset into the `_LDR_DATA_TABLE_ENTRY` it means that when we attempt to cast a pointer to a `LIST_ENTRY` as a `_LDR_DATA_TABLE_ENTRY` we are off by some amount (in this case 2 * PVOID = 8 bytes). The following diagram attempts to explain the issue.

![](https://i.imgur.com/1G44FsV.png)

One hack might be to create our own custom struct that starts at the offset, but this would be madness when dealing with more than a few types... instead we have... **shifted pointers**!

#### IDA Shifted Pointers
IDA has a simple concept (with some insane syntax) to deal with this issue called a [shifted pointer](https://www.hex-rays.com/products/ida/support/idadoc/1695.shtml). When assigning a type to `LIST_ENTRY` the shifted pointer syntax can be used to tell IDA that it is actually a pointer inside a larger struct with an offset. 

```C
_LIST_ENTRY *__shifted(_LDR_DATA_TABLE_ENTRY,8) pListEntry
```

![](https://i.imgur.com/hvVJkP1.png)

#### References 
- PE Header notes [yates](https://web.archive.org/web/20181222200541/http://www.woodmann.com/yates/PE_Information/PE_Notes.pdf)
- Geoff Chappell [LDR_DATA_TABLE_ENTRY](https://www.geoffchappell.com/studies/windows/km/ntoskrnl/inc/api/ntldr/ldr_data_table_entry.htm)
- [Understanding LIST_ENTRY Lists and Its Importance in Operating Systems] (https://www.codeproject.com/Articles/800404/Understanding-LIST-ENTRY-Lists-and-Its-Importance)
- MSDN [PEB_LDR_DATA](https://learn.microsoft.com/en-us/windows/win32/api/winternl/ns-winternl-peb_ldr_data)
- IDA Tricks [CONTAINING_RECORD macro](https://www.hex-rays.com/products/decompiler/manual/tricks.shtml#03)
- IDA docs [Shifted Pointers](https://www.hex-rays.com/products/ida/support/idadoc/1695.shtml)

#### Thanks 
- [@yates82](https://twitter.com/yates82)
- [@printup](https://github.com/anthonyprintup)
- [@dodo](https://twitter.com/dodo_sec)

Thanks to everyone who helped me figure this out for once and for all! 

